In [1]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
userlog = pd.read_csv('user_logs.csv')

In [3]:
# Set datetime object as the index
userlog = userlog.set_index(userlog.datetime)
# Drop the original datetime column
userlog.drop(columns='datetime',inplace=True)

In [4]:
# Set 'datetime' as a datetime index.
userlog.index = pd.to_datetime(userlog.index)

In [5]:
userlog.head()

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,,
2018-01-26 09:55:03,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,/,1,97.105.19.61
2018-01-26 09:56:02,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,java-ii,1,97.105.19.61
2018-01-26 09:56:05,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,java-ii/object-oriented-programming,1,97.105.19.61
2018-01-26 09:56:06,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,slides/object_oriented_programming,1,97.105.19.61
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,javascript-i/conditionals,2,97.105.19.61


## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [6]:
# Drop 'deleted_at' column
userlog.drop(columns='deleted_at', inplace=True)

In [7]:
userlog.dropna(inplace=True)

In [8]:
userlog

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip
datetime,,,,,,,,,,,
2018-01-26 09:55:03,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,/,1,97.105.19.61
2018-01-26 09:56:02,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,1,97.105.19.61
2018-01-26 09:56:05,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii/object-oriented-programming,1,97.105.19.61
2018-01-26 09:56:06,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides/object_oriented_programming,1,97.105.19.61
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/conditionals,2,97.105.19.61
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/personal-site,64,71.150.217.33
2021-04-21 16:42:02,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/mapbox-api,64,71.150.217.33
2021-04-21 16:42:09,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/ajax/weather-map,64,71.150.217.33


In [9]:
userlog['path_split'] = userlog.path.str.split('/')

In [10]:
userlog['lesson'] = [mod[-1] for mod in userlog.path_split]

In [11]:
userlog

,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip,path_split,lesson
datetime,,,,,,,,,,,,,
2018-01-26 09:55:03,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,/,1,97.105.19.61,"[, ]",
2018-01-26 09:56:02,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,1,97.105.19.61,[java-ii],java-ii
2018-01-26 09:56:05,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii/object-oriented-programming,1,97.105.19.61,"[java-ii, object-oriented-programming]",object-oriented-programming
2018-01-26 09:56:06,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides/object_oriented_programming,1,97.105.19.61,"[slides, object_oriented_programming]",object_oriented_programming
2018-01-26 09:56:24,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i/conditionals,2,97.105.19.61,"[javascript-i, conditionals]",conditionals
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/personal-site,64,71.150.217.33,"[jquery, personal-site]",personal-site
2021-04-21 16:42:02,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/mapbox-api,64,71.150.217.33,"[jquery, mapbox-api]",mapbox-api
2021-04-21 16:42:09,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,jquery/ajax/weather-map,64,71.150.217.33,"[jquery, ajax, weather-map]",weather-map


In [56]:
userlog.groupby(['cohort_name','lesson']).agg('sum')


cohort_id  \
cohort_name lesson                                                       
Andromeda                                                        36394   
            %252e%252e%255c                                         31   
            ..%255c                                                 31   
            ..%2f                                                   62   
            ..%5c                                                   31   
...                                                                ...   
Zion        website-status-page                                    435   
            where                                                 4437   
            workbook                                               174   
            working-in-teams                                      1015   
            working-with-data-types-operators-and-variables      10324   

                                                             program_id  \
cohort_name lesson                                                        
Andromeda                                                          2348   
            %252e%252e%255c                                           2   
            ..%255c                                                   2   
            ..%2f                                                     4   
            ..%5c                                                     2   
...                                                                 ...   
Zion        website-status-page                                      30   
            where                                                   306   
            workbook                                                 12   
            working-in-teams                                         70   
            working-with-data-types-operators-and-variables         712   

                                                             user_id  
cohort_name lesson                                                    
Andromeda                                                     465267  
            %252e%252e%255c                                      373  
            ..%255c                                              373  
            ..%2f                                                746  
            ..%5c                                                373  
...                                                              ...  
Zion        website-status-page                                 5049  
            where                                              50973  
            workbook                                            1977  
            working-in-teams                                   11807  
            working-with-data-types-operators-and-variables   119474  

[10968 rows x 3 columns]

In [21]:
userlog.groupby('lesson').agg('max')


,cohort_id,cohort_name,slack,start_date,end_date,created_at,updated_at,program_id,path,user_id,ip,path_split
lesson,,,,,,,,,,,,
,139,Zion,#zion,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,4,/,981,99.98.36.168,"[, ]"
%252e%252e%255c,31,Andromeda,#andromeda,2019-03-18,2019-07-30,2019-03-18 20:35:06,2019-03-18 20:35:06,2,examples/css/%252e%252e%255c,373,97.105.19.58,"[examples, css, %252e%252e%255c]"
',132,Staff,#kalypso,2020-11-02,2021-05-04,2020-11-02 20:43:58,2020-11-02 20:43:58,2,java-iii/mvc/java-beans/',793,99.137.94.0,"[java-iii, mvc, java-beans, ']"
..%255c,31,Andromeda,#andromeda,2019-03-18,2019-07-30,2019-03-18 20:35:06,2019-03-18 20:35:06,2,examples/css/..%255c,373,97.105.19.58,"[examples, css, ..%255c]"
..%2f,31,Andromeda,#andromeda,2019-03-18,2019-07-30,2019-03-18 20:35:06,2019-03-18 20:35:06,2,examples/css/..%2f,373,97.105.19.58,"[examples, css, ..%2f]"
...,...,...,...,...,...,...,...,...,...,...,...,...
www.followthemoney.org,133,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,3,appendix/open_data/www.followthemoney.org,835,99.162.246.69,"[appendix, open_data, www.followthemoney.org]"
www.google.com,52,Teddy,#teddy,2019-11-04,2020-04-17,2019-11-04 18:27:07,2019-11-04 18:27:07,2,javascript-i/introduction/working-with-data-ty...,528,99.88.62.179,"[javascript-i, introduction, working-with-data..."
www.opensecrets.org,59,Darden,#darden,2020-07-13,2021-01-12,2020-07-13 18:32:19,2020-07-13 18:32:19,3,appendix/www.opensecrets.org,695,99.162.246.69,"[appendix, www.opensecrets.org]"


In [126]:
pd.set_option('display.max_rows',3000)

In [143]:
cohorts  = userlog.groupby(['cohort_name']).path.value_counts() 
cohorts

cohort_name  path                     
Andromeda    /                            1174
             toc                           638
             javascript-i                  516
             spring                        487
             java-iii                      433
                                          ... 
Zion         uploads/598dc43df39e2.jpg       1
             web-design/intro                1
             web-design/ui/visuals           1
             web-design/ux/layout            1
             web-design/ux/purpose           1
Name: path, Length: 13565, dtype: int64

TypeError: cannot unpack non-iterable int object

In [71]:

userlog.groupby(['cohort_name','lesson']).sum().user_id.nlargest(5)

cohort_name  lesson  
Darden                   2060163
             overview    1753903
Staff                    1575781
Jupiter      toc         1427523
                         1292946
Name: user_id, dtype: int64

In [100]:
userlog.groupby(['cohort_name','path','lesson']).sum().user_id.nlargest(10)

cohort_name  path                      lesson           
Darden       /                                              2060163
Staff        /                                              1575781
Jupiter      toc                       toc                  1427523
             /                                              1292946
Ganymede     /                                              1096214
Fortuna      /                                              1079685
Curie        /                                              1005677
Bayes        /                                               935461
Apex         search/search_index.json  search_index.json     912541
Hyperion     /                                               906143
Name: user_id, dtype: int64

In [99]:
userlog.groupby(['cohort_name','path']).sum().user_id.nsmallest(30)

cohort_name  path                                                                  
Hampton      0_Classification_Algorithms                                               1
             Dataset_Challenge                                                         1
             Module_6_Classification                                                   1
             appendix/angular/models                                                   1
             appendix/extra-challenges/css-practice                                    1
             appendix/further-reading                                                  1
             appendix/further-reading/javascript/array-splice                          1
             appendix/further-reading/javascript/hoisting                              1
             appendix/further-reading/javascript/hoisting.html                         1
             appendix/further-reading/mysql/host-wildcards                             1
             appendix/git/

In [52]:
 maxi = userlog.groupby(['cohort_name','lesson']).sum()

In [53]:
maxi.cohort_id.nlargest(10)

cohort_name  lesson      
Staff                        177520
Darden                       175820
             overview        150745
Easley                       148295
             overview        142842
Kalypso                      123948
Marco        toc             122850
             javascript-i    122445
Jupiter      toc             115692
Fortuna                      108014
Name: cohort_id, dtype: int64

In [54]:
maxi.user_id.nlargest(10)

cohort_name  lesson    
Darden                     2060163
             overview      1753903
Staff                      1575781
Jupiter      toc           1427523
                           1292946
Ganymede                   1096214
Fortuna                    1079685
Curie                      1005677
Bayes                       935461
Curie        1-overview     922994
Name: user_id, dtype: int64

In [57]:
maxi2 = userlog.groupby(['cohort_name','lesson']).agg('sum')

In [58]:
maxi2.cohort_id.nlargest(10)

cohort_name  lesson      
Staff                        177520
Darden                       175820
             overview        150745
Easley                       148295
             overview        142842
Kalypso                      123948
Marco        toc             122850
             javascript-i    122445
Jupiter      toc             115692
Fortuna                      108014
Name: cohort_id, dtype: int64

In [59]:
maxi2.user_id.nlargest(10)

cohort_name  lesson    
Darden                     2060163
             overview      1753903
Staff                      1575781
Jupiter      toc           1427523
                           1292946
Ganymede                   1096214
Fortuna                    1079685
Curie                      1005677
Bayes                       935461
Curie        1-overview     922994
Name: user_id, dtype: int64

In [90]:
userlog.groupby(['cohort_name','lesson']).sum().user_id.nsmallest(5)

cohort_name  lesson                     
Hampton      0_Classification_Algorithms    1
             13                             1
             58a217a705bde.jpg              1
             Dataset_Challenge              1
             Module_6_Classification        1
Name: user_id, dtype: int64

In [95]:
userlog.ip.value_counts().nsmallest(25)

69.169.52.37       1
107.77.64.50       1
137.254.7.169      1
136.50.61.52       1
107.77.218.64      1
107.150.31.108     1
212.102.40.36      1
70.121.228.51      1
192.24.18.31       1
24.227.144.238     1
76.193.64.119      1
173.239.232.136    1
107.77.218.210     1
172.58.107.139     1
172.58.67.243      1
37.60.189.18       1
107.77.218.99      1
110.77.182.87      1
172.58.96.115      1
172.58.107.190     1
174.244.81.249     1
172.58.108.187     1
172.58.103.199     1
172.58.109.220     1
99.203.90.180      1
Name: ip, dtype: int64

In [98]:
userlog.ip.value_counts().nlargest(5)

97.105.19.58       268648
97.105.19.61        60530
192.171.117.210      9124
71.150.217.33        6791
76.185.145.231       4754
Name: ip, dtype: int64